# The ``Grid`` Object

Here we show how to instantiate a ``Grid`` object and use it to explore a grid file.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from unyt import angstrom

The ``Grid`` object needs a file to load, these are HDF5 files that are available through the ``synthesizer-download`` command line tool (for more details see [the introduction to grids](grids.rst). By default, once downloaded these files are stored in the ``GRID_DIR`` directory. The default location of this directory is platform dependent, but the location can be found by import it and printing it.

In [ ]:
from synthesizer import GRID_DIR

print(GRID_DIR)

This directory can be overriden by setting the ``SYNTHESIZER_GRID_DIR`` environment variable.

Assuming the grid file is in the default location, all we need to do is pass the name of the grid we want to load to the ``Grid`` constructor. Note that the name of the grid can include the extension or not. If the extension is not included, it is assumed to be ``"hdf5"``. 

Here we will load the test grid (a simplified BPASS 2.2.1 grid).

In [ ]:
from synthesizer import Grid

grid = Grid("test_grid.hdf5")

If we are loading a grid from a different location we can just pass that path to the ``grid_dir`` argument.

In [ ]:
import os
from shutil import copyfile

# Copy the grid file to the stated directory to
# demonstrate the grid_dir argument
copyfile(
    os.path.join(GRID_DIR, "test_grid.hdf5"),
    os.path.join("../../../tests/test_grid", "test_grid.hdf5"),
)

In [ ]:
grid = Grid("test_grid.hdf5", grid_dir="../../../tests/test_grid")

## Printing a summary of the `Grid`

We can have a look at what the loaded grid contains by simply printing the grid. 

In [ ]:
print(grid)

In this instance, its a stellar grid with the `incident` spectrum defined by the `axes_values`, `ages` and `metallicities`. The grid also contains some useful quantites like the photon rate (`log10_specific_ionising_luminosity`) available for fully ionising hydrogen and helium. 

Since this grid is a cloudy processed grid, there are additional spectra or line data that are available to extract or manipulate. These include (but not limited to)
- `spectra`
    - `nebular`: is the nebular continuum (including line emission) predicted by the photoionisation model
    - `linecont`: this is the line contribution to the spectrum
    - `transmitted`: this is the incident spectra that is transmitted through the gas in the photoionisation modelling; it has zero flux at shorter wavelength of the lyman-limit
    - `wavelength`: the wavelength covered
- `lines`
    - `id`: line id, this is the same as used in cloudy (see [Linelist generation](https://github.com/synthesizer-project/grid-generation/tree/main/src/synthesizer_grids/cloudy/create_linelist))
    - `luminosity`: the luminosity of the line
    - `nebular_continuum`: the underlying nebular continuum at the line
    - `transmitted`: this is the transmitted luminosity at the line
    - `wavelength`:  the wavelength of the line

A similar structure is also followed for AGN grids, where the `axes` could either be described by `mass` (black hole mass), `acretion_rate_eddington` (the accretion rate normalised to the eddington limit for the mass), `cosine_inclination` (cosine value describing the inclination of the AGN), or the `temperature` (blackbody temperature of the big bump component), `alpha-ox` (X-ray to UV ratio) , `alpha-uv` (low-energy slope of the big bump component), `alpha-x` (slope of the X-ray component).

## Limiting the ``Grid``

A `Grid` can also take various arguments to limit the size of the grid, e.g. by isolating the `Grid` to a wavelength region of interest. This is particularly useful when making a large number of spectra from a high resolution `Grid`, where the memory footprint can become large.

### Passing a wavelength array

If you only care about a grid of specific wavelength values, you can pass this array and the `Grid` will automatically be interpolated onto the new wavelength array using [SpectRes](https://github.com/ACCarnall/SpectRes).

In [ ]:
# Define a new set of wavelengths
new_lams = np.logspace(2, 5, 1000) * angstrom

# Create a new grid
grid = Grid("test_grid", new_lam=new_lams)
print(grid.shape)

### Passing wavelength limits

If you don't want to modify the underlying grid resolution, but only care about a specific wavelength range, you can pass limits to truncate the grid at.

In [ ]:
# Create a new grid
grid = Grid("test_grid", lam_lims=(10**3 * angstrom, 10**4 * angstrom))
print(grid.shape)

## Plot a single grid point

We can plot the spectra at the location of a single point in our grid. First, we choose some age and metallicity.

In [ ]:
# Return to the unmodified grid
grid = Grid("test_grid")

log10age = 6.0  # log10(age/yr)
Z = 0.01  # metallicity

We then get the index location of that grid point for this age and metallicity

In [ ]:
grid_point = grid.get_grid_point(log10ages=log10age, metallicity=Z)

We can then loop over the available spectra (contained in `grid.spec_names`) and plot

In [ ]:
for spectra_type in grid.available_spectra:
    # Get `Sed` object
    sed = grid.get_sed_at_grid_point(grid_point, spectra_type=spectra_type)

    # Mask zero valued elements
    mask = sed.lnu > 0
    plt.plot(
        np.log10(sed.lam[mask]),
        np.log10(sed.lnu[mask]),
        lw=1,
        alpha=0.8,
        label=spectra_type,
    )

plt.legend(fontsize=8, labelspacing=0.0)
plt.xlim(2.3, 8)
plt.ylim(19, 25)
plt.xlabel(r"$\rm log_{10}(\lambda/\AA)$")
plt.ylabel(r"$\rm log_{10}(L_{\nu}/erg\ s^{-1}\ Hz^{-1} M_{\odot}^{-1})$")

## Plot ionising luminosities

We can also plot properties over the entire age and metallicity grid, such as the ionising luminosity. 

In the examples below we plot ionising luminosities for HI and HeII

In [ ]:
fig, ax = grid.plot_specific_ionising_lum(ion="HI")

In [ ]:
fig, ax = grid.plot_specific_ionising_lum(ion="HeII")